In [1]:
import pandas as pd
import numpy as np
import json
import re
import csv
import pickle
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
class wordRemover():
    def __init__(self, word):
        self.word = word
        
    def removeWord(self, listOfWords):
        if self.word in listOfWords:
            index = listOfWords.index(self.word)
            del listOfWords[index]
        return listOfWords

In [3]:
METADATA_PATH = "/Users/soumava/Genre-Prediction/data/movie.metadata.tsv"
PLOT_SUMMARIES_PATH = "/Users/soumava/Genre-Prediction/data/plot_summaries.txt"

In [4]:
metadata = pd.read_csv(METADATA_PATH, sep = '\t', header = None)
metadata.columns = ["movie_id",1,"movie_name",3,4,5,6,7,"genre"]

plots = []
with open(PLOT_SUMMARIES_PATH, 'r') as f:
    reader = csv.reader(f, dialect='excel-tab') 
    for row in tqdm(reader):
        plots.append(row)

movie_id = []
plot = []

# extract movie Ids and plot summaries
for i in tqdm(plots):
    movie_id.append(i[0])
    plot.append(i[1])

# create dataframe
movies = pd.DataFrame({'movie_id': movie_id, 'plot': plot})

# change datatype of 'movie_id'
metadata['movie_id'] = metadata['movie_id'].astype(str)

# merge meta with movies
movies = pd.merge(movies, metadata[['movie_id', 'movie_name', 'genre']], on = 'movie_id')

genres = [] 

# extract genres
for i in movies['genre']: 
    genres.append(list(json.loads(i).values())) 

# add to 'movies' dataframe  
movies['genre_new'] = genres
genres = movies['genre_new'].values.tolist()
binarizer = MultiLabelBinarizer()
y_data = binarizer.fit_transform(genres)

42303it [00:01, 40016.62it/s]
100%|██████████| 42303/42303 [00:00<00:00, 1264406.55it/s]


In [5]:
counts = []
categories = binarizer.classes_
for i in range(categories.shape[0]):
    counts.append((categories[i], np.sum(y_data[:,i])))
df_stats = pd.DataFrame(counts, columns=['genre', '#movies'])

x=df_stats[df_stats['#movies']<10]
genres_to_remove = x['genre'].values.tolist()

for word in genres_to_remove:
    movies['genre_new'] = movies['genre_new'].apply(wordRemover(word).removeWord)
    
movies_new = movies[~(movies['genre_new'].str.len() == 0)]

In [6]:
movies_new.shape

(41787, 5)

In [9]:
new_genres = movies_new['genre_new'].values.tolist()
binarizer1 = MultiLabelBinarizer()
y_data1 = binarizer1.fit_transform(new_genres)

In [10]:
y_data1.shape

(41787, 261)

In [11]:
counts1 = []
categories1 = binarizer1.classes_
for i in range(categories1.shape[0]):
    counts1.append((categories1[i], np.sum(y_data1[:,i])))
df_stats = pd.DataFrame(counts1, columns=['genre', '#movies'])

In [13]:
df_stats.sort_values(by=['#movies'], ascending=True)

,genre,#movies
195,Psycho-biddy,10
257,World History,10
30,Bengali Cinema,10
105,Family & Personal Relationships,10
22,Archaeology,10
55,Cold War,10
115,Filipino,10
103,Extreme Sports,10
122,Foreign legion,10
42,Buddy cop,11
